# Sales Forecasting and Customer Segmentation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('ggplot')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Load the datasets
retail_df = pd.read_csv('retail_sales_data.csv')
reviews_df = pd.read_csv('product_reviews.csv')

# Convert date column to datetime
retail_df['date'] = pd.to_datetime(retail_df['date'])

# Add temporal features
retail_df['year'] = retail_df['date'].dt.year
retail_df['month'] = retail_df['date'].dt.month
retail_df['day'] = retail_df['date'].dt.day
retail_df['day_of_week'] = retail_df['date'].dt.dayofweek
retail_df['is_weekend'] = retail_df['day_of_week'].isin([5, 6]).astype(int)
retail_df['quarter'] = retail_df['date'].dt.quarter

# Handle missing values
print("Handling missing values...")
# Fill missing weather with mode
retail_df['weather'] = retail_df['weather'].fillna(retail_df['weather'].mode()[0])
# Fill missing promotion with 'None'
retail_df['promotion'] = retail_df['promotion'].fillna('None')
# Fill missing special_event with False
retail_df['special_event'] = retail_df['special_event'].fillna(False)
# Fill missing dominant_age_group with mode
retail_df['dominant_age_group'] = retail_df['dominant_age_group'].fillna(retail_df['dominant_age_group'].mode()[0])
# Fill missing numerical values with median
for col in ['num_customers', 'total_sales', 'online_sales', 'in_store_sales', 'avg_transaction', 'return_rate']:
    retail_df[col] = retail_df[col].fillna(retail_df[col].median())

print("Missing values after imputation:")
print(retail_df.isnull().sum())

# Feature Engineering
print("\nPerforming feature engineering...")

# Create lag features for time series
def create_lag_features(df, group_cols, target_col, lag_periods):
    df_copy = df.copy()
    for lag in lag_periods:
        df_copy[f'{target_col}_lag_{lag}'] = df_copy.groupby(group_cols)[target_col].shift(lag)
    return df_copy

# Create rolling window features
def create_rolling_features(df, group_cols, target_col, windows, functions):
    df_copy = df.copy()
    for window in windows:
        for func in functions:
            df_copy[f'{target_col}_roll_{window}_{func.__name__}'] = df_copy.groupby(group_cols)[target_col].transform(
                lambda x: x.shift(1).rolling(window=window, min_periods=1).agg(func))
    return df_copy

# Sort by date for proper time series analysis
retail_df = retail_df.sort_values(['store_id', 'category', 'date'])

# Create lag features for total_sales (1, 7, and 14 days)
retail_df = create_lag_features(retail_df, ['store_id', 'category'], 'total_sales', [1, 7, 14])

# Create rolling window features (7, 14, and 30 days with mean and std)
retail_df = create_rolling_features(retail_df, ['store_id', 'category'], 'total_sales', [7, 14, 30], [np.mean, np.std])

# Create interaction features
retail_df['price_per_customer'] = retail_df['total_sales'] / retail_df['num_customers']
retail_df['online_ratio'] = retail_df['online_sales'] / retail_df['total_sales']
retail_df['weekend_promotion'] = retail_df['is_weekend'] * (retail_df['promotion'] != 'None').astype(int)

# Drop rows with NaN values created by lag features
retail_df = retail_df.dropna()

print("Feature engineering completed. New shape:", retail_df.shape)

# Part 1: Sales Forecasting
print("\n=== Sales Forecasting Model ===")

# Prepare data for forecasting
# We'll forecast total_sales for each store and category combination

# Select features for forecasting
forecast_features = [
    'month', 'day', 'day_of_week', 'is_weekend', 'quarter',
    'weather', 'promotion', 'special_event', 'dominant_age_group',
    'num_customers', 'online_sales', 'in_store_sales', 'avg_transaction',
    'total_sales_lag_1', 'total_sales_lag_7', 'total_sales_lag_14',
    'total_sales_roll_7_mean', 'total_sales_roll_14_mean', 'total_sales_roll_30_mean',
    'total_sales_roll_7_std', 'total_sales_roll_14_std', 'total_sales_roll_30_std',
    'price_per_customer', 'online_ratio', 'weekend_promotion'
]

# Target variable
target = 'total_sales'

# Prepare categorical features
cat_features = ['weather', 'promotion', 'special_event', 'dominant_age_group']
num_features = [f for f in forecast_features if f not in cat_features]

# Create a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ])

# Create a time series split for validation
tscv = TimeSeriesSplit(n_splits=3)

# Function to evaluate forecasting model
def evaluate_forecast_model(X, y, model, cv):
    mae_scores = []
    rmse_scores = []
    r2_scores = []
    
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mae_scores.append(mean_absolute_error(y_test, y_pred))
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
        r2_scores.append(r2_score(y_test, y_pred))
    
    return {
        'MAE': np.mean(mae_scores),
        'RMSE': np.mean(rmse_scores),
        'R2': np.mean(r2_scores)
    }

# Select a sample store and category for demonstration
store_id = 'store_1'
category = 'Electronics'
sample_data = retail_df[(retail_df['store_id'] == store_id) & (retail_df['category'] == category)]

X = sample_data[forecast_features]
y = sample_data[target]

# Create and evaluate XGBoost model
xgb_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42))
])

print(f"Training forecasting model for {store_id}, {category}...")
forecast_metrics = evaluate_forecast_model(X, y, xgb_model, tscv)
print("Forecasting Model Metrics:")
for metric, value in forecast_metrics.items():
    print(f"{metric}: {value:.4f}")

# Feature importance
xgb_model.fit(X, y)
feature_importance = xgb_model.named_steps['regressor'].feature_importances_

# Get feature names after one-hot encoding
preprocessor.fit(X)
feature_names = (
    num_features +
    list(preprocessor.named_transformers_['cat'].get_feature_names_out(cat_features))
)

# Create feature importance DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(15))
plt.title(f'Top 15 Feature Importance for Sales Forecasting ({store_id}, {category})')
plt.tight_layout()
plt.savefig('forecast_feature_importance.png')
plt.close()

# Visualize actual vs predicted values
X_train, X_test = X.iloc[:-30], X.iloc[-30:]
y_train, y_test = y.iloc[:-30], y.iloc[-30:]

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test.values, label='Actual', marker='o')
plt.plot(y_test.index, y_pred, label='Predicted', marker='x')
plt.title(f'Actual vs Predicted Sales ({store_id}, {category})')
plt.xlabel('Date Index')
plt.ylabel('Total Sales')
plt.legend()
plt.tight_layout()
plt.savefig('forecast_actual_vs_predicted.png')
plt.close()

# Part 2: Customer Segmentation
print("\n=== Customer Segmentation Analysis ===")

# Aggregate data at the store level for segmentation
store_features = retail_df.groupby('store_id').agg({
    'total_sales': 'mean',
    'online_sales': 'mean',
    'in_store_sales': 'mean',
    'num_customers': 'mean',
    'avg_transaction': 'mean',
    'return_rate': 'mean',
    'online_ratio': 'mean',
    'price_per_customer': 'mean',
    'is_weekend': 'mean'  # Proportion of weekend sales
}).reset_index()

# Calculate additional KPIs
store_features['online_to_instore_ratio'] = store_features['online_sales'] / store_features['in_store_sales']

# Prepare data for clustering
X_cluster = store_features.drop('store_id', axis=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# Determine optimal number of clusters using silhouette score
silhouette_scores = []
K = range(2, 6)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Plot silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(K, silhouette_scores, 'bo-')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score Method For Optimal k')
plt.grid(True)
plt.savefig('optimal_clusters.png')
plt.close()

# Choose optimal number of clusters
optimal_k = K[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_k}")

# Apply K-means clustering with optimal k
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
store_features['cluster'] = kmeans.fit_predict(X_scaled)

# Analyze clusters
cluster_analysis = store_features.groupby('cluster').agg({
    'total_sales': 'mean',
    'online_sales': 'mean',
    'in_store_sales': 'mean',
    'num_customers': 'mean',
    'avg_transaction': 'mean',
    'return_rate': 'mean',
    'online_ratio': 'mean',
    'price_per_customer': 'mean',
    'online_to_instore_ratio': 'mean'
}).reset_index()

print("\nCluster Analysis:")
print(cluster_analysis)

# Visualize clusters
plt.figure(figsize=(12, 8))
for i in range(optimal_k):
    cluster_data = store_features[store_features['cluster'] == i]
    plt.scatter(
        cluster_data['online_ratio'],
        cluster_data['price_per_customer'],
        label=f'Cluster {i}',
        s=100
    )
    
    # Add store labels
    for _, row in cluster_data.iterrows():
        plt.annotate(row['store_id'], (row['online_ratio'], row['price_per_customer']), 
                     fontsize=9, ha='center')

plt.xlabel('Online Sales Ratio')
plt.ylabel('Price Per Customer')
plt.title('Store Clusters by Online Ratio and Price Per Customer')
plt.legend()
plt.grid(True)
plt.savefig('store_clusters.png')
plt.close()

# Radar chart for cluster profiles
def radar_chart(cluster_data, cluster_col='cluster'):
    # Select features for the radar chart
    features = ['total_sales', 'online_ratio', 'num_customers', 
                'avg_transaction', 'return_rate', 'price_per_customer']
    
    # Number of variables
    N = len(features)
    
    # Create a figure
    fig = plt.figure(figsize=(12, 8))
    
    # Normalize the data for radar chart
    scaler = StandardScaler()
    cluster_data_scaled = pd.DataFrame(
        scaler.fit_transform(cluster_data[features]),
        columns=features
    )
    cluster_data_scaled[cluster_col] = cluster_data[cluster_col]
    
    # Compute the angle for each feature
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Close the loop
    
    # Initialize the plot
    ax = plt.subplot(111, polar=True)
    
    # Draw one axis per variable and add labels
    plt.xticks(angles[:-1], features, size=12)
    
    # Draw the y-axis labels (0 to 2)
    ax.set_rlabel_position(0)
    plt.yticks([0, 1, 2], ["0", "1", "2"], color="grey", size=10)
    plt.ylim(0, 2)
    
    # Plot each cluster
    for i in range(optimal_k):
        cluster_values = cluster_data_scaled[cluster_data_scaled[cluster_col] == i][features].mean().values.flatten().tolist()
        cluster_values += cluster_values[:1]  # Close the loop
        ax.plot(angles, cluster_values, linewidth=2, linestyle='solid', label=f'Cluster {i}')
        ax.fill(angles, cluster_values, alpha=0.1)
    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title('Cluster Profiles', size=15)
    
    return fig

# Create radar chart
radar_fig = radar_chart(store_features)
plt.savefig('cluster_profiles_radar.png')
plt.close()

# Create cluster profile descriptions
cluster_profiles = []
for i in range(optimal_k):
    cluster_data = cluster_analysis[cluster_analysis['cluster'] == i]
    
    # Determine key characteristics
    if cluster_data['online_ratio'].values[0] > cluster_analysis['online_ratio'].mean():
        online_status = "High online sales ratio"
    else:
        online_status = "Low online sales ratio"
        
    if cluster_data['price_per_customer'].values[0] > cluster_analysis['price_per_customer'].mean():
        price_status = "High average spend per customer"
    else:
        price_status = "Low average spend per customer"
        
    if cluster_data['return_rate'].values[0] > cluster_analysis['return_rate'].mean():
        return_status = "High return rate"
    else:
        return_status = "Low return rate"
    
    # Create profile
    profile = f"Cluster {i}: {online_status}, {price_status}, {return_status}"
    cluster_profiles.append(profile)
    
    # List stores in this cluster
    stores = store_features[store_features['cluster'] == i]['store_id'].tolist()
    stores_str = ", ".join(stores)
    cluster_profiles.append(f"   Stores: {stores_str}")

print("\nCluster Profiles:")
for profile in cluster_profiles:
    print(profile)

# Return key results
print("\nSales Forecasting and Customer Segmentation Analysis Completed")
print(f"Forecasting model for {store_id}, {category} achieved R² of {forecast_metrics['R2']:.4f}")
print(f"Stores were segmented into {optimal_k} distinct clusters")
print("Visualizations saved as PNG files")

# Return the dataframes for further analysis
store_features.head(), forecast_metrics